### Meme SIR Project

Tracking the spread of memes

Katie Thai-Tang, Emma Pan, Erika Serna

In [ ]:
"""Notes
run stochastic function many times
for loop
alpha parameter to make see thru

resistance is second wave

In [ ]:
"""
Notes 10/19/17
First thing we should do is change the values in the working SIR model so it is applicable to our model.
Next we should change the update function so it uses whole numbers instead of fractions.

If we do the qestion about dead meme threshold, we should decide on a point ot cut off data.
"""

In [2]:
%matplotlib inline

from modsim import *

ModuleNotFoundError: No module named 'modsim'

In [ ]:
init = State(S=349, I=1, R=0)
# S=People who haven't heard of the meme, I=People using the meme, R=People who are tired of the meme
init /= sum(init)

In [ ]:
#This is the part we change to be stochastic
tc = 3    #time between contact in days
tr = 4    #time between recoveries in days

In [ ]:
beta = 1/tc   #contact rate per day
gamma = 1/tr   #recovery rate per day

In [ ]:
#Change Values for new project
def make_system(beta, gamma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(S=89, I=1, R=0)
    init /= sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma)

In [ ]:
#test make_system

system = make_system(beta, gamma)

In [1]:
#Change this function, maybe not fraction?
#Change Values for new project
def update1(state, system):
    """Update the SIR model.
    
    state: State with variables S, I, R
    system: System with beta and gamma
    
    returns: State object
    """
    s, i, r = state

    infected = system.beta * i * s    
    recovered = system.gamma * i
    
    s -= infected
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)

In [ ]:
#test update1
state = update1(init, system)
state

### TimeSeries

In [ ]:
#change Values for new project
def run_simulation(system, update_func):
    """Runs a simulation of the system.
    
    Add three Series objects to the System: S, I, R
    
    system: System object
    update_func: function that updates state
    """
    S = TimeSeries()
    I = TimeSeries()
    R = TimeSeries()

    state = system.init
    t0 = system.t0
    S[t0], I[t0], R[t0] = state
    
    for t in linrange(system.t0, system.t_end):
        state = update_func(state, system)
        S[t+1], I[t+1], R[t+1] = state
    
    system.S = S
    system.I = I
    system.R = R

In [ ]:
#test run_simulation TIMESERIES

tc = 3      # time between contacts in days 
tr = 4      # recovery time in days

beta = 1 / tc      # contact rate in per day
gamma = 1 / tr     # recovery rate in per day

system = make_system(beta, gamma)
run_simulation(system, update1)

def plot_results(S, I, R):
    """Plot the results of a SIR model.
    
    S: TimeSeries
    I: TimeSeries
    R: TimeSeries
    """
    plot(S, '--', color='blue', label='Susceptible')
    plot(I, '-', color='red', label='Infected')
    plot(R, ':', color='green', label='Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of population')
    
plot_results(system.S, system.I, system.R)
savefig('Meme-fig01.pdf')

### DataFrame

In [5]:
#change values for new prject
def run_simulation(system, update_func):
    """Runs a simulation of the system.
    
    Add a DataFrame to the System: results
    
    system: System object
    update_func: function that updates state
    """
    frame = DataFrame(columns=system.init.index)
    frame.loc[system.t0] = system.init
    
    for t in linrange(system.t0, system.t_end):
        frame.loc[t+1] = update_func(frame.loc[t], system)
    
    system.results = frame

In [ ]:
#test run_simulation DATAFRAME

tc = 3      # time between contacts in days 
tr = 4      # recovery time in days

beta = 1 / tc      # contact rate in per day
gamma = 1 / tr     # recovery rate in per day

sir = make_system(beta, gamma)
run_simulation(system, update1)
system.results.head()

frame = system.results
plot_results(frame.S, frame.I, frame.R)

### Metrics

In [ ]:
#Change values for new project
def calc_total_infected(system):
    """Fraction of population infected during the simulation.
    
    system: System object with results.
    
    returns: fraction of population
    """
    frame = system.results
    return frame.S[system.t0] - frame.S[system.t_end]

In [ ]:
#Test calc_total_infected
system.beta = 0.333
system.gamma = 0.25
run_simulation(system, update1)
print(system.beta, system.gamma, calc_total_infected(system))